# Classificação do Adult Census Income com Rede Neural (MLPClassifier)

Este notebook demonstra como utilizar uma rede neural (MLPClassifier do scikit-learn) para resolver o problema de classificação do dataset Adult Census Income. O fluxo abrange:
- Carregamento e pré-processamento dos dados
- Divisão dos dados em treino e teste
- Transformação dos dados (padronização e One-Hot Encoding)
- Treinamento do modelo
- Avaliação do modelo

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

## Carregamento e Pré-processamento do Dataset

Neste passo, carregamos o dataset 'Adult' do OpenML, substituímos os valores "?" por NaN e removemos as linhas com dados faltantes. Em seguida, separamos as features da variável target e mapeamos os rótulos para valores numéricos.

In [ ]:
# Carrega o dataset 'adult' do OpenML
adult = fetch_openml('adult', version=2, as_frame=True)
df = adult.frame

# Substitui valores "?" por NaN e remove linhas com dados faltantes
df = df.replace('?', np.nan).dropna()

# Separa features (variáveis preditoras) e target (rótulo)
X = df.drop(columns='class')
y = df['class'].map({'<=50K': 0, '>50K': 1})

## Divisão dos Dados em Treino e Teste

Utilizamos o `train_test_split` para dividir os dados em 80% para treino e 20% para teste, mantendo a proporção das classes com o parâmetro `stratify`.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## Pré-processamento dos Dados: Padronização e One-Hot Encoding

Primeiro, padronizamos as features numéricas utilizando o `StandardScaler`. Em seguida, aplicamos One-Hot Encoding às features categóricas usando o `OneHotEncoder`.

Por fim, concatenamos as features transformadas em um único array para o treinamento do modelo.

In [ ]:
# Define as colunas numéricas e categóricas
numeric_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = [col for col in X.columns if col not in numeric_features]

# Padroniza as colunas numéricas
scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train[numeric_features])
X_test_num = scaler.transform(X_test[numeric_features])

# Aplica One-Hot Encoding nas colunas categóricas
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_train_cat = encoder.fit_transform(X_train[categorical_features])
X_test_cat = encoder.transform(X_test[categorical_features])

# Concatena as features numéricas e categóricas
X_train_transformed = np.concatenate([X_train_num, X_train_cat], axis=1)
X_test_transformed = np.concatenate([X_test_num, X_test_cat], axis=1)

## Definição e Treinamento do Modelo de Rede Neural

Utilizamos o `MLPClassifier` do scikit-learn para definir uma rede neural com duas camadas ocultas (64 e 32 neurônios, respectivamente). Configuramos a função de ativação `relu`, o solver `adam` e ativamos o early stopping para evitar overfitting.

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(64, 32),
                      activation='relu',
                      solver='adam',
                      max_iter=200,
                      random_state=42,
                      early_stopping=True)

# Treina o modelo
model.fit(X_train_transformed, y_train)

## Avaliação do Modelo

Realizamos a avaliação do modelo utilizando o conjunto de teste. Calculamos a acurácia e geramos um relatório de classificação com métricas detalhadas (precisão, recall, f1-score) para cada classe.

In [ ]:
# Realiza as previsões no conjunto de teste
y_pred = model.predict(X_test_transformed)

# Calcula a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia no conjunto de teste: {accuracy:.4f}")

# Gera o relatório de classificação
print(classification_report(y_test, y_pred))